<span style="color:blue; font-size:30px; line-height:40px; font-family: Calibri;">Project 3: Data wrangling report</span>

<img src="../images/weratedogs.png" alt="drawing" width="500" style="float:left"/>

<span style="color:blue; font-size:26px; line-height:40px; font-family: Calibri;">Project Overview</span>

The goal of the project was to demonstrate data wrangling and analysis skills obtained from the course.

The dataset to be wrangled is the tweet archive of Twitter user [@dog_rates](https://twitter.com/dog_rates), also known as [WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs). WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. Dog owners upload a picture of their dog, writes a funny comment and give their own dog a rating out of 10. Owners almost always give their dogs a rating greater than 10. e.g 11/10, 12/10, 13/10, it's the nature of the website. WeRateDogs has over 4 million followers and has received international media coverage. It's only a bit of fun, and brings great joy when browsing the cutest dog images !


### Data wrangling consist of 3 main steps:
>#### **Gather -> Assess -> Clean**

<span style="color:blue; font-size:26px; line-height:40px; font-family: Calibri;">Wrangling Step 1: Data gathering</span>

Real-world data rarely comes clean, especially when it comes from the internet where users enter data into free text fields that are optional, as it's the case with this twitter account.

During the data gathering step, twitter data needed to be downloaded **automaticaly** from the internet using a variety of sources and techniques, with the help of Python and its libraries.

3 data gathering strategies was demonstrated:

### 1. Twitter archive - Automated downloaded from an URL

WeRateDogs downloaded their Twitter archive and sent it to Udacity via email exclusively to use in this project. This archive contains basic tweet data (tweet ID, timestamp, text, etc.) for a sample of 2356 tweets with ratings, for a period between Nov 2015 - July 2017.
Data was automatically downloaded from the Udacity website using `pd.read_csv` and passing the URL as input.

### 2. Use twitter API 'tweepy v2' to retrieve additional data

For each tweet in the twitter archive from step 1, the retweet and like count was retrieved as minimum requirement for the project

The latest version of the tweepy v2 API was used to retrieve the data, which offered more flexibility and data availability, although it took some time to figure out how to use it. I initially tried to download data like geo data, country, place, additional metrics etc, as these could make interesting analytics, however this data was never filled.

Each tweet's entire JSON response was dumped to a txt file, each tweet on its own line

The .txt file was read line by line, and the most useful data was added to a pandas dataframe. Some API data were already present in twitter archive from step 1, however might be useful to compliment missing or corrupt data potentially. 

Some API errors occurred which was dumped to txt log file. Some tweets were no longer availabe in the API.


### 3. Use the Requests library to download the tweet breed predictions (image_predictions.tsv)

Udacity provided a file that used the dog images to predict the breed of the dog.
Each tweet have a URL to the image that gave the best breed predictions, 3 predictions each with it's confidence level and if the prediction is actually a valid dog breed.

This file (image_predictions.tsv), is hosted on Udacity's servers and was downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv


### 4. Download dog profile images (optional extra step)

Image urls containing `.jpg` retrieved from API in step 2 was used to download the first dog image for each tweet from the internet, as additional practice. I had in mind for visualization to show a dog profile card of the most liked breeds and dogs.

<span style="color:blue; font-size:26px; line-height:40px; font-family: Calibri;">Wrangling Step 2: Assessing</span>

**Note:** 
Data obtained are indeed very untidy. The requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues to demonstrate data wrangling skills.
Even though I went beyond this requirement, still not all issues are assessed and cleaned to perfection.
The data were assessed and cleaned, upon which the analysis and visualizations will be based.

Assessing was done using first visual inspection, followed by programmatic inspection for each of the 3 datasets acquired in step 1.
Visual inspection was done by manually inspecting the data after displaying a sample of 10-20 records, using `.sample()`
Programmatic inspection was done using .`info(), nunqiue(), unique(), .duplicated()`. Deeper inspection was done where problems were spotted, using filtering and query techniques.

During the assessment stage, the 'what' is documented, describing only the **observed** problem. The problems were organized by quality vs tidiness issues.

Quality issues were sub organized by completeness, validity, accuracy and consistency, and sorted in order of importance, the same order that will be followed during the cleaning process.

<img src="../images/assessment.png" alt="assessment" width="600" style="float:left"/>

<span style="color:blue; font-size:26px; line-height:40px; font-family: Calibri;">Wrangling Step 3: Cleaning</span>

First, copies were made of each dataset, which was further used to perform the cleaning actions on.

Cleaning was done following the framework **Define -> Code -> Test** for each issue found during assessing one-by-one

Example:

<img src="../images/cleanexample.png" alt="cleanexample" width="900" style="float:left"/>

At the end of the cleaning step, we have a new master twitter archive dataset exported as `.csv` file, containing no duplicate columns, with the best data merged from all 3 datasets.
Only original tweets with images were kept.

<img src="../images/masterstructure.png" alt="datastructure" width="400" style="float:left"/>